## Методы оптимизации. Составление целевых функций

### Задача о раскрое

Сталепрокатное предприятие выпускает стандартные стальные листы размерами 1500×6000 мм.
Ежемесячный объем заказов на листы нестандартных размеров приведён ниже:
500×600 — 3000, 1000×400 — 2000, 1500×1000 — 1500
Необходимо составить ЦФ для нахождения наименее затратного раскроя листов.
** и составить алгоритм

Имеется m вариантов раскроя, т.е. известны величины $a_{ij} (i= \overline{1,m}, j=\overline{1,3})$, определяющие количество единиц j-x изделий при i-м способе раскроя одного листа и остаток $a_i$.

Пусть $x_i$ - количество листов, раскраиваемые i-м вариантом $(i=\overline{1,m})$

Количество изделий 1-го, 2-го и 3-го вида с учетом условия количества:

$$a_{11}x_1+...+a_{i1}x_i+...+a_{m1}x_m = 3000$$

$$a_{12}x_1+...+a_{i2}x_i+...+a_{m2}x_m = 2000$$

$$a_{13}x_1+...+a_{i3}x_i+...+a_{m3}x_m = 1500$$


Целевая функция:
$$F = \sum_{i=1}^{m}x_ia_i \rightarrow min$$

$$a_{1j}x_1+...+a_{ij}x_i+...+a_{mj}x_m = k_j$$

где $k_j$ - необходимое количество раскроенных листов


Задача относится к NP-полным, при которой с усложением условий перебор всех вариантов становится недоступен. Существует несколько способов поиска оптимального и приближенного оптимального решения: с помощью динамического программирования, использования эвристик, генетических алгоритмов, градиентных алгоритмов 

Для данной постановки задачи в качестве наиболее простого способа можно воспользоваться методом поиска решения, известным, как **"method of Divine Inspiration"** - т.е. с помощью поиска очевидных закономерностей, ведущим к оптимальному решению.

Для этого можно составить наиболее очевидные эффективные карты раскроя, игнорируя менее перспективные комбинации, т.к. в задаче сохраняется принцип зеркальности и небольшая размерность вектора конфигураций деталей - 3, в результате чего решение задачи сводится к построению функции Лагранжа и решению системы уравнений 

Конфигурация раскроя в общем виде:

  Варианты раскроя   | $x_{j_1}$  | $x_{j_2}$  | $x_{j_3}$  | Площадь отходов $a$ | Кол-во листов $x_{max}$
      ---------------|------------|------------|------------|---------------------|---------------
  Граничные условия: | 3000       | 2000       | 1500       | $a \rightarrow min$ | $x \rightarrow min$
      $m_1$          | 30         | 0          | 0          | 0                   | 100
      $m_2$          | 0          | 21         | 0          | 0.6                 | 96
      $m_3$          | 0          | 0          | 6          | 0                   | 250
      $m_4$          | 10         | 15         | 0          | 0                   | 134
      ...            | ...        | ...        | ...        | ...                 | ...
      $m_n$          |$x_{j_{1n}}$|$x_{j_{2n}}$|$x_{j_{3n}}$| $a_{n}$             | $x_{max_n}$


In [167]:
import numpy as np

A1 = np.array([[30., 0., 0., 10.], [0., 21., 0., 15.], [0., 0., 6., 0.]])
b1 = np.array([3000., 2000., 1500.])

A2 = np.array([[30., 0., 10.], [0., 0., 15.], [0., 6., 0.]])
b2 = np.array([3000., 2000., 1500.])

C1 = np.around(np.linalg.lstsq(A1, b1)[0], 3)
C2 = np.around(np.linalg.lstsq(A2, b2)[0], 3)
print(C1)
print(C2)

[ 79.161  50.583 250.     62.517]
[ 55.556 250.    133.333]


C:\Users\Nickel\anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  if __name__ == '__main__':
C:\Users\Nickel\anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  # Remove the CWD from sys.path while we load stuff.


Получено 2 решения: в кадом варианте ответ следует округлить до целого числа вверх, дробная часть ответа указывает на превышение нормы по распилу листов нестандартного размера, которое по условию задачи можно отнести к отходам. Также в первом решении имеется неучтенный остаток, получаемый для каждого раскроенного листа - 2-го элемента в соответсвтии с конфигурацией раскроя, который составляет 100 * 6000 * 1e-6 = 0.6 м$^2$

Во втором случае количество остатка в виде неформата равняется 0, фактический остаток складывается из нереализованных заготовок. Посчитаем остаток

In [176]:
# площадь листа в м2
S = 1500 * 6000 * 1e-6

# Остатки
# 
losses1 = (np.ceil(C1) - C1) * S
losses2 = (np.ceil(C2) - C2) * S

print(f"Потребность в стандартных листах для 1-го решения: {np.sum(np.ceil(C1))}")
print(f"Потребность в стандартных листах для 2-го решения: {np.sum(np.ceil(C2))}\n")

print(f"Остатки листов для 1-го решения (м^2): {losses1}")
print(f"Остатки листов для 2-го решения (м^2): {losses2}\n")

loss11 = round(np.sum(losses1),2)
loss12 = loss11 + np.ceil(C1[1]) * 0.6
loss2 = round(np.sum(losses2),2)

print(f"Суммарные остатки листов для 1-го решения (м^2): {loss11}, с учетом неформата: {loss12}")
print(f"Суммарные остатки листов для 2-го решения (м^2): {loss2}, с учетом неформата: {loss2}")

Потребность в стандартных листах для 1-го решения: 444.0
Потребность в стандартных листах для 2-го решения: 440.0

Остатки листов для 1-го решения (м^2): [7.551 3.753 0.    4.347]
Остатки листов для 2-го решения (м^2): [3.996 0.    6.003]

Суммарные остатки листов для 1-го решения (м^2): 15.65, с учетом неформата: 46.25
Суммарные остатки листов для 2-го решения (м^2): 10.0, с учетом неформата: 10.0


В итоге получили оптимальное решение, при котором объем заказов нестандартных размеров обеспечивается за счет раскроя 440 стандартных листов, с нулевыми отходами и остатком: 20 листов (500x600) и 10 листов (400x1000), общей площадью 10 м$^2$

### Задача о курьерах

Пусть есть компания, занимающаяся доставкой Belivery Club.
Есть N зон, в которых есть спрос на курьеров в каждый час. Внутри дня в разные часы спрос различен. Каждой зоне i в час h необходимо $n_{hi}$ курьеров. Для каждой зоны есть штраф из-за недостаточно 1 курьера - $s_i$. Так же у каждого курьера есть предпочтения относительно часов и зон работы. Необходимо распределить M курьеров таким образом, чтобы суммарный штраф был минимальный.
(описать формулу в общем случае, т.е. без каких-либо числовых данных)

Пусть $x_{ih}^{k_{ih}}$ = 1, если k-курьер с предпочтением к i зоне в час h работает в этой зоне в час h

Тогда в одной зоне в час h работает курьеров: $$\sum_{k_{ih}=1}^{M} x_{ih}^{k_{ih}}$$

Штраф в час в этой зоне

$$f_{ih} =\displaystyle\begin{cases}0, \text{if} \sum_{k_{ih}=1}^{M} x_{ih}^{k_{ih}} - n_{ih} >0 \\
s_i \cdot (\sum_{k_{ih}=1}^{M} x_{ih}^{k_{ih}} - n_{ih}), \text{in other case} \end{cases}$$ 


Штраф за день $$\sum_{h=1}^{24}f_{ih}$$

Штраф по всем зонам в течении дня:


$$\sum_{i=1}^{N}\sum_{h=1}^{24}f_{ih}$$

Целевая функция:

$$\sum_{i=1}^{N}\sum_{h=1}^{24}f_{ih} \rightarrow min $$
